In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv -O leads_coring.csv


--2025-10-21 02:41:58--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: 'leads_coring.csv'

     0K .......... .......... .......... .......... .......... 63%  731K 0s
    50K .......... .......... ........                        100% 4,14M=0,08s

2025-10-21 02:41:59 (1,02 MB/s) - 'leads_coring.csv' saved [80876/80876]



In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('leads_coring.csv')
df

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1
...,...,...,...,...,...,...,...,...,...
1457,referral,manufacturing,1,NaN,self_employed,north_america,4,0.53,1
1458,referral,technology,3,65259.0,student,europe,2,0.24,1
1459,paid_ads,technology,1,45688.0,student,north_america,3,0.02,1
1460,referral,NaN,5,71016.0,self_employed,north_america,0,0.25,1


In [4]:
df.converted.isnull().sum()

0

In [45]:
len(df)-df.notnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [46]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [5]:
df.dtypes[df.dtypes == 'object']

lead_source          object
industry             object
employment_status    object
location             object
dtype: object

In [6]:
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)
categorical_columns

['lead_source', 'industry', 'employment_status', 'location']

In [7]:
df.head().T

,0,1,2,3,4
lead_source,paid_ads,social_media,events,paid_ads,referral
industry,NaN,retail,healthcare,retail,education
number_of_courses_viewed,1,1,5,2,3
annual_income,79450.0,46992.0,78796.0,83843.0,85012.0
employment_status,unemployed,employed,unemployed,NaN,self_employed
location,south_america,south_america,australia,australia,europe
interaction_count,4,1,3,1,3
lead_score,0.94,0.8,0.69,0.87,0.62
converted,1,0,1,0,1


In [8]:
numeric_columns = list(df.dtypes[df.dtypes != 'object'].index)
numeric_columns

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score',
 'converted']

In [9]:
df[categorical_columns].isna().sum()

lead_source          128
industry             134
employment_status    100
location              63
dtype: int64

In [10]:
df[categorical_columns]= df[categorical_columns].fillna('NA')
df[categorical_columns].isna().sum()

lead_source          0
industry             0
employment_status    0
location             0
dtype: int64

In [11]:
df[numeric_columns].isnull().sum()

number_of_courses_viewed      0
annual_income               181
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [12]:
df[numeric_columns] = df[numeric_columns].fillna(0)
df[numeric_columns].isnull().sum()

number_of_courses_viewed    0
annual_income               0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [13]:
df['industry'].nunique()

8

## Question 1
What is the most frequent observation (mode) for the column industry?

In [14]:
df.industry.value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

Split the data
Split your data in train/val/test sets with 60%/20%/20% distribution.
Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
Make sure that the target value converted is not in your dataframe.

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [17]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [18]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [19]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

In [20]:
numeric_columns.pop()
numeric_columns

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

## Question 2.
 Biggest correlation (1 point)

In [21]:
df_full_train[numeric_columns].corrwith(df_full_train.converted).abs()

number_of_courses_viewed    0.442068
annual_income               0.029612
interaction_count           0.378482
lead_score                  0.225641
dtype: float64

In [22]:
import pprint
correlation_matrix = df_full_train[numeric_columns].corr()
styled_corr = correlation_matrix.style.background_gradient(cmap='coolwarm', vmin=-1, vmax=1).format(precision=3)

styled_corr

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000,0.012,-0.044,0.009
annual_income,0.012,1.000,0.012,0.021
interaction_count,-0.044,0.012,1.000,0.025
lead_score,0.009,0.021,0.025,1.000


## Question 3
 - Calculate the mutual information score between converted and other categorical variables in the dataset. Use the training set only.
- Round the scores to 2 decimals using round(score, 2).

Which of these variables has the biggest mutual information score?

- industry
- location
- lead_source
- employment_status

In [23]:
from sklearn.metrics import mutual_info_score

In [24]:
df_full_train

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
1066,social_media,manufacturing,2,44403.0,self_employed,australia,1,0.71,0
638,events,retail,3,38048.0,student,north_america,6,0.97,1
799,social_media,education,2,71399.0,NA,europe,1,0.51,1
380,referral,education,2,47912.0,employed,australia,1,0.04,0
303,paid_ads,healthcare,1,34806.0,employed,europe,4,0.32,1
...,...,...,...,...,...,...,...,...,...
1130,events,manufacturing,1,57039.0,employed,south_america,2,0.30,0
1294,events,healthcare,2,56185.0,student,south_america,2,0.44,0
860,paid_ads,manufacturing,1,56402.0,student,north_america,1,0.02,0
1459,paid_ads,technology,1,45688.0,student,north_america,3,0.02,1


In [25]:
def mutual_info_converted_score(series):
    return mutual_info_score(series, df_full_train.converted)

mi = df_full_train[categorical_columns].apply(mutual_info_converted_score)
round(mi.sort_values(ascending=False),2)

lead_source          0.03
employment_status    0.01
industry             0.01
location             0.00
dtype: float64

In [37]:
from sklearn.feature_extraction import DictVectorizer

In [38]:
numeric_columns

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [39]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical_columns + numeric_columns].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical_columns + numeric_columns].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [40]:
from sklearn.linear_model import LogisticRegression

In [41]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [42]:
model.intercept_[0]

-0.0691472802783609

In [43]:
model.coef_[0].round(2)

array([-0.  , -0.01,  0.03,  0.  ,  0.01, -0.1 , -0.02,  0.05, -0.02,
       -0.01, -0.  , -0.01, -0.03, -0.02,  0.31,  0.05,  0.02, -0.01,
       -0.01, -0.12,  0.08, -0.03,  0.  , -0.01, -0.01, -0.01,  0.01,
        0.01, -0.03, -0.03,  0.45])

In [44]:
y_pred = model.predict_proba(X_val)[:, 1]


In [45]:
CONVERTED_decision = (y_pred >= 0.5)

In [46]:
# accuracy of validation set
full_accuracy = round((y_val == CONVERTED_decision).mean(),4)
full_accuracy

0.6997

In [53]:
dv.get_feature_names_out()

array(['annual_income', 'employment_status=NA',
       'employment_status=employed', 'employment_status=self_employed',
       'employment_status=student', 'employment_status=unemployed',
       'industry=NA', 'industry=education', 'industry=finance',
       'industry=healthcare', 'industry=manufacturing', 'industry=other',
       'industry=retail', 'industry=technology', 'interaction_count',
       'lead_score', 'lead_source=NA', 'lead_source=events',
       'lead_source=organic_search', 'lead_source=paid_ads',
       'lead_source=referral', 'lead_source=social_media', 'location=NA',
       'location=africa', 'location=asia', 'location=australia',
       'location=europe', 'location=middle_east',
       'location=north_america', 'location=south_america',
       'number_of_courses_viewed'], dtype=object)

In [54]:
model.coef_[0].round(3)

array([-0.   , -0.015,  0.034,  0.003,  0.012, -0.103, -0.025,  0.049,
       -0.02 , -0.013, -0.003, -0.009, -0.032, -0.016,  0.311,  0.051,
        0.02 , -0.012, -0.012, -0.115,  0.08 , -0.03 ,  0.004, -0.011,
       -0.011, -0.006,  0.008,  0.006, -0.033, -0.025,  0.454])

In [55]:
features_dict = dict(zip(dv.get_feature_names_out(), model.coef_[0].round(3)) )
features_dict

{'annual_income': -0.0,
 'employment_status=NA': -0.015,
 'employment_status=employed': 0.034,
 'employment_status=self_employed': 0.003,
 'employment_status=student': 0.012,
 'employment_status=unemployed': -0.103,
 'industry=NA': -0.025,
 'industry=education': 0.049,
 'industry=finance': -0.02,
 'industry=healthcare': -0.013,
 'industry=manufacturing': -0.003,
 'industry=other': -0.009,
 'industry=retail': -0.032,
 'industry=technology': -0.016,
 'interaction_count': 0.311,
 'lead_score': 0.051,
 'lead_source=NA': 0.02,
 'lead_source=events': -0.012,
 'lead_source=organic_search': -0.012,
 'lead_source=paid_ads': -0.115,
 'lead_source=referral': 0.08,
 'lead_source=social_media': -0.03,
 'location=NA': 0.004,
 'location=africa': -0.011,
 'location=asia': -0.011,
 'location=australia': -0.006,
 'location=europe': 0.008,
 'location=middle_east': 0.006,
 'location=north_america': -0.033,
 'location=south_america': -0.025,
 'number_of_courses_viewed': 0.454}

In [56]:
all_features = dv.get_feature_names_out().tolist()
all_features

['annual_income',
 'employment_status=NA',
 'employment_status=employed',
 'employment_status=self_employed',
 'employment_status=student',
 'employment_status=unemployed',
 'industry=NA',
 'industry=education',
 'industry=finance',
 'industry=healthcare',
 'industry=manufacturing',
 'industry=other',
 'industry=retail',
 'industry=technology',
 'interaction_count',
 'lead_score',
 'lead_source=NA',
 'lead_source=events',
 'lead_source=organic_search',
 'lead_source=paid_ads',
 'lead_source=referral',
 'lead_source=social_media',
 'location=NA',
 'location=africa',
 'location=asia',
 'location=australia',
 'location=europe',
 'location=middle_east',
 'location=north_america',
 'location=south_america',
 'number_of_courses_viewed']

In [57]:
def log_reg_without_feature(features_to_use, df_train, df_val, y_val):
    smaill_features = features_to_use

    dicts_train_small = df_train[smaill_features].to_dict(orient='records')
    dicts_val_small = df_val[smaill_features].to_dict(orient='records')

    dv_small = DictVectorizer(sparse=False)
    dv_small.fit(dicts_train_small)

    X_val_small = dv_small.transform(dicts_val_small)
    X_train_small = dv_small.transform(dicts_train_small)

    model_small = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model_small.fit(X_train_small, y_train)

    y_pred = model_small.predict_proba(X_val_small)[:, 1]
    CONVERTED_decision = (y_pred >= 0.5)
    accuracy = round((y_val == CONVERTED_decision).mean(),4)
    
    return accuracy

In [58]:
import pprint

all_columns = numeric_columns+categorical_columns
accuracy_dict = {}

for feature in all_columns:
    All_features = all_columns.copy()
    All_features.remove(feature)
    accuracy = log_reg_without_feature(All_features, df_train, df_val, y_val)
    print(f'{feature}\taccuracy={accuracy}\tfull_accuracy{full_accuracy}')
    accuracy_dict[feature]=float(round(full_accuracy-accuracy,6))

sorted_items = sorted(accuracy_dict.items(), key=lambda item: item[1])
pprint.pprint(sorted_items)

number_of_courses_viewed	accuracy=0.5563	full_accuracy0.6997
annual_income	accuracy=0.8532	full_accuracy0.6997
interaction_count	accuracy=0.5563	full_accuracy0.6997
lead_score	accuracy=0.7065	full_accuracy0.6997
lead_source	accuracy=0.7031	full_accuracy0.6997
industry	accuracy=0.6997	full_accuracy0.6997
employment_status	accuracy=0.6962	full_accuracy0.6997
location	accuracy=0.7099	full_accuracy0.6997
[('annual_income', -0.1535),
 ('location', -0.0102),
 ('lead_score', -0.0068),
 ('lead_source', -0.0034),
 ('industry', 0.0),
 ('employment_status', 0.0035),
 ('number_of_courses_viewed', 0.1434),
 ('interaction_count', 0.1434)]


## Question 6
 - Now let's train a regularized logistic regression.
- Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
- Train models using all the features as in Q4.
- Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

In [59]:
dicts_full_train = df_full_train[categorical_columns + numeric_columns].to_dict(orient='records')
dicts_train = df_train[categorical_columns + numeric_columns].to_dict(orient='records')

In [60]:
dv = DictVectorizer(sparse=False)
X_full_train = dv.fit_transform(dicts_full_train)


In [61]:
dv_ = DictVectorizer(sparse=False)
X_train = dv_.fit_transform(dicts_train)

In [62]:
y_full_train = df_full_train.converted.values


In [63]:
from sklearn.metrics import accuracy_score


In [64]:
dicts_train = df_train[categorical_columns + numeric_columns].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(dicts_train)

val_dict = df_val[categorical_columns + numeric_columns].to_dict(orient='records')
X_val = dv.transform(val_dict)

def reg_log_regression(C_parameter, X_train, y_train, y_val):

    model = LogisticRegression(solver='liblinear',max_iter=2000, C=C_parameter,  random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict_proba(X_val)[:, 1]
    CONVERTED_decision = (y_pred >= 0.5)
    
    return (CONVERTED_decision == y_val).mean()


for c in [0.01, 0.1, 1, 10, 100]:
    accuracy  = reg_log_regression(c, X_train, y_train, y_val)
    print(f'c = {c}\t accuracy ={round(accuracy ,10)} ')

c = 0.01	 accuracy =0.6996587031 
c = 0.1	 accuracy =0.6996587031 
c = 1	 accuracy =0.6996587031 
c = 10	 accuracy =0.6996587031 
c = 100	 accuracy =0.6996587031 


In [65]:
dicts_train = df_train[categorical_columns + numeric_columns].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(dicts_train)

# Validation data
val_dict = df_val[categorical_columns + numeric_columns].to_dict(orient='records')
X_val = dv.transform(val_dict) # Create X_val here, just once!

# --- Define a cleaner function ---
def reg_log_regression(C_parameter, X_train, y_train, X_val, y_val): # Now accepts X_val
    model = LogisticRegression(solver='lbfgs', max_iter=2000, C=C_parameter, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val) # Use .predict() for direct accuracy check
    
    return accuracy_score(y_val, y_pred)


# --- Loop and print results ---
for c in [0.01, 0.1, 1, 10, 100]:
    # Pass the pre-made X_val to the function
    accuracy = reg_log_regression(c, X_train, y_train, X_val, y_val)
    print(f'c = {c}\t accuracy = {round(accuracy, 6)}')

c = 0.01	 accuracy = 0.812287
c = 0.1	 accuracy = 0.843003
c = 1	 accuracy = 0.856655
c = 10	 accuracy = 0.853242
c = 100	 accuracy = 0.853242


In [66]:
import pickle


In [67]:
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

In [69]:
var = df_full_train[:1]
var

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
1066,social_media,manufacturing,2,44403.0,self_employed,australia,1,0.71,0


In [76]:
X_full_train[:1]

array([[4.4403e+04, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
        7.1000e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        2.0000e+00]])

In [78]:
 model.predict(X_full_train[:1])[0]

1